# Workflow Server Example

Example demonstrating how to use the WorkflowServer with event streaming.

This example shows how to:
1. Create workflows that emit streaming events
2. Set up the server with event streaming support
3. Register workflows
4. Run the server
5. Make HTTP requests to execute workflows
6. Stream real-time events from running workflows using the /events endpoint

In [ ]:
%pip install llama-agents-server

## Run the server in background

In [4]:
import subprocess
import time

# Start server in background using subprocess
server_process = subprocess.Popen(["python", "server.py"])
time.sleep(2)  # Wait for server to start
print(f"Server started with PID: {server_process.pid}")

INFO:     Started server process [27939]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Server started with PID: 27939


## Interact with the server

In [5]:
# Hit the health endpoint to see the server is up and running
!curl http://localhost:8000/health

INFO:     127.0.0.1:65182 - "GET /health HTTP/1.1" 200 OK
{"status":"healthy","loaded_workflows":0,"active_workflows":0,"idle_workflows":0}

In [6]:
# List available workflows
!curl http://localhost:8000/workflows

INFO:     127.0.0.1:65184 - "GET /workflows HTTP/1.1" 200 OK
{"workflows":["greeting","math","processing"]}

In [7]:
# Run greeting workflow
!curl -X POST http://localhost:8000/workflows/greeting/run \
  -H "Content-Type: application/json" \
  -d '{"kwargs": {"name": "Alice"}}'

INFO:     127.0.0.1:65190 - "POST /workflows/greeting/run HTTP/1.1" 200 OK
{"handler_id":"3podAAguhA","workflow_name":"greeting","run_id":"KD7Y2S0e0h","error":null,"result":{"value":{"result":"Hello, Alice!"},"qualified_name":"workflows.events.StopEvent","type":"StopEvent","types":null},"status":"completed","started_at":"2026-01-29T22:36:48.596733+00:00","updated_at":"2026-01-29T22:36:49.500898+00:00","completed_at":"2026-01-29T22:36:49.500898+00:00"}

In [8]:
# Run math workflow
!curl -X POST http://localhost:8000/workflows/math/run \
  -H "Content-Type: application/json" \
  -d '{"kwargs": {"a": 10, "b": 5, "operation": "multiply"}}'

INFO:     127.0.0.1:65196 - "POST /workflows/math/run HTTP/1.1" 200 OK
{"handler_id":"kITlArjbWm","workflow_name":"math","run_id":"t3jXb1nGtV","error":null,"result":{"value":{"result":{"a":10,"b":5,"operation":"multiply","result":50}},"qualified_name":"workflows.events.StopEvent","type":"StopEvent","types":null},"status":"completed","started_at":"2026-01-29T22:36:56.341122+00:00","updated_at":"2026-01-29T22:36:56.341812+00:00","completed_at":"2026-01-29T22:36:56.341812+00:00"}

In [36]:
%%bash
# Run workflow with nowait
handler_id=$(curl -sX POST http://localhost:8000/workflows/math/run-nowait \
  -H "Content-Type: application/json" \
  -d '{"kwargs": {"a": 100, "b": 25, "operation": "divide"}}' | jq -r ".handler_id" )
printf "Got handler id: ${handler_id}\n\n"

# Wait for the workflow to run in background
sleep 1

# Fetch the result asynchronously
curl -s http://localhost:8000/results/${handler_id}

Got handler id: vuyyRzWikr
{"result":{"a":100,"b":25,"operation":"divide","result":4.0}}

In [9]:
%%bash
# Stream events from workflow

# 1. Run workflow with nowait
handler_id=$(curl -sX POST http://localhost:8000/workflows/greeting/run-nowait \
  -H "Content-Type: application/json" \
  -d '{"kwargs": {"name": "Async User"}}' | jq -r ".handler_id" )
printf "Got handler id: ${handler_id}\n\n"

# Wait for the workflow to run in background
sleep 1

printf "Streaming events...\n"
# 2. Stream events using Server-Sent Events using SSE format
curl -s http://localhost:8000/events/$handler_id?sse=true


printf "\nFinal result:\n"
# 3. Get the final result after events complete
curl -s http://localhost:8000/results/$handler_id

INFO:     127.0.0.1:65202 - "POST /workflows/greeting/run-nowait HTTP/1.1" 200 OK
Got handler id: gbqSi1fzGJ



Streaming events...
INFO:     127.0.0.1:65204 - "GET /events/gbqSi1fzGJ?sse=true HTTP/1.1" 200 OK
data: {"value":{"sequence":0},"qualified_name":"__main__.StreamEvent","type":"StreamEvent","types":null}

data: {"value":{"sequence":1},"qualified_name":"__main__.StreamEvent","type":"StreamEvent","types":null}

data: {"value":{"sequence":2},"qualified_name":"__main__.StreamEvent","type":"StreamEvent","types":null}

data: {"value":{"result":"Hello, Async User!"},"qualified_name":"workflows.events.StopEvent","type":"StopEvent","types":null}


Final result:
INFO:     127.0.0.1:65206 - "GET /results/gbqSi1fzGJ HTTP/1.1" 200 OK
{"handler_id":"gbqSi1fzGJ","workflow_name":"greeting","run_id":"M5AtcoxBty","error":null,"result":"Hello, Async User!","status":"completed","started_at":"2026-01-29T22:37:02.375925+00:00","updated_at":"2026-01-29T22:37:03.279696+00:00","completed_at":"2026-01-29T22:37:03.279696+00:00"}